In [1]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as num

In [2]:
# -*- coding: utf-8 -*-
import random

def cop_kmeans(dataset, k, ml=[], cl=[],
               initialization='kmpp',
               max_iter=300, tol=1e-4):

    ml, cl = transitive_closure(ml, cl, len(dataset))
    ml_info = get_ml_info(ml, dataset)
    tol = tolerance(tol, dataset)

    centers = initialize_centers(dataset, k, initialization)

    for _ in range(max_iter):
        clusters_ = [-1] * len(dataset)
        for i, d in enumerate(dataset):
            indices, _ = closest_clusters(centers, d)
            counter = 0
            if clusters_[i] == -1:
                found_cluster = False
                while (not found_cluster) and counter < len(indices):
                    index = indices[counter]
                    if not violate_constraints(i, index, clusters_, ml, cl):
                        found_cluster = True
                        clusters_[i] = index
                        for j in ml[i]:
                            clusters_[j] = index
                    counter += 1

                if not found_cluster:
                    return None, None

        clusters_, centers_ = compute_centers(clusters_, dataset, k, ml_info)
        shift = sum(l2_distance(centers[i], centers_[i]) for i in range(k))
        if shift <= tol:
            break

        centers = centers_

    return clusters_, centers_

def l2_distance(point1, point2):
    return sum([(float(i)-float(j))**2 for (i, j) in zip(point1, point2)])

# taken from scikit-learn (https://goo.gl/1RYPP5)
def tolerance(tol, dataset):
    n = len(dataset)
    dim = len(dataset[0])
    averages = [sum(dataset[i][d] for i in range(n))/float(n) for d in range(dim)]
    variances = [sum((dataset[i][d]-averages[d])**2 for i in range(n))/float(n) for d in range(dim)]
    return tol * sum(variances) / dim

def closest_clusters(centers, datapoint):
    distances = [l2_distance(center, datapoint) for
                 center in centers]
    return sorted(range(len(distances)), key=lambda x: distances[x]), distances

def initialize_centers(dataset, k, method):
    if method == 'random':
        ids = list(range(len(dataset)))
        random.shuffle(ids)
        return [dataset[i] for i in ids[:k]]

    elif method == 'kmpp':
        chances = [1] * len(dataset)
        centers = []

        for _ in range(k):
            chances = [x/sum(chances) for x in chances]
            r = random.random()
            acc = 0.0
            for index, chance in enumerate(chances):
                if acc + chance >= r:
                    break
                acc += chance
            centers.append(dataset[index])

            for index, point in enumerate(dataset):
                cids, distances = closest_clusters(centers, point)
                chances[index] = distances[cids[0]]

        return centers

def violate_constraints(data_index, cluster_index, clusters, ml, cl):
    for i in ml[data_index]:
        if clusters[i] != -1 and clusters[i] != cluster_index:
            return True

    for i in cl[data_index]:
        if clusters[i] == cluster_index:
            return True

    return False

def compute_centers(clusters, dataset, k, ml_info):
    cluster_ids = set(clusters)
    k_new = len(cluster_ids)
    id_map = dict(zip(cluster_ids, range(k_new)))
    clusters = [id_map[x] for x in clusters]

    dim = len(dataset[0])
    centers = [[0.0] * dim for i in range(k)]

    counts = [0] * k_new
    for j, c in enumerate(clusters):
        for i in range(dim):
            centers[c][i] += dataset[j][i]
        counts[c] += 1

    for j in range(k_new):
        for i in range(dim):
            centers[j][i] = centers[j][i]/float(counts[j])

    if k_new < k:
        ml_groups, ml_scores, ml_centroids = ml_info
        current_scores = [sum(l2_distance(centers[clusters[i]], dataset[i])
                              for i in group)
                          for group in ml_groups]
        group_ids = sorted(range(len(ml_groups)),
                           key=lambda x: current_scores[x] - ml_scores[x],
                           reverse=True)

        for j in range(k-k_new):
            gid = group_ids[j]
            cid = k_new + j
            centers[cid] = ml_centroids[gid]
            for i in ml_groups[gid]:
                clusters[i] = cid

    return clusters, centers

def get_ml_info(ml, dataset):
    flags = [True] * len(dataset)
    groups = []
    for i in range(len(dataset)):
        if not flags[i]: continue
        group = list(ml[i] | {i})
        groups.append(group)
        for j in group:
            flags[j] = False

    dim = len(dataset[0])
    scores = [0.0] * len(groups)
    centroids = [[0.0] * dim for i in range(len(groups))]

    for j, group in enumerate(groups):
        for d in range(dim):
            for i in group:
                centroids[j][d] += dataset[i][d]
            centroids[j][d] /= float(len(group))

    scores = [sum(l2_distance(centroids[j], dataset[i])
                  for i in groups[j])
              for j in range(len(groups))]

    return groups, scores, centroids

def transitive_closure(ml, cl, n):
    ml_graph = dict()
    cl_graph = dict()
    for i in range(n):
        ml_graph[i] = set()
        cl_graph[i] = set()

    def add_both(d, i, j):
        d[i].add(j)
        d[j].add(i)

    for (i, j) in ml:
        add_both(ml_graph, i, j)

    def dfs(i, graph, visited, component):
        visited[i] = True
        for j in graph[i]:
            if not visited[j]:
                dfs(j, graph, visited, component)
        component.append(i)

    visited = [False] * n
    for i in range(n):
        if not visited[i]:
            component = []
            dfs(i, ml_graph, visited, component)
            for x1 in component:
                for x2 in component:
                    if x1 != x2:
                        ml_graph[x1].add(x2)
    for (i, j) in cl:
        add_both(cl_graph, i, j)
        for y in ml_graph[j]:
            add_both(cl_graph, i, y)
        for x in ml_graph[i]:
            add_both(cl_graph, x, j)
            for y in ml_graph[j]:
                add_both(cl_graph, x, y)

    for i in ml_graph:
        for j in ml_graph[i]:
            if j != i and j in cl_graph[i]:
                raise Exception('inconsistent constraints between %d and %d' %(i, j))

    return ml_graph, cl_graph




In [ ]:
#sample use of cop-kmeans
import numpy 
input_matrix = numpy.random.rand(60, 80)*100
must_link = [(0, 10), (0, 20), (0, 30)]
cannot_link = [(1, 10), (2, 10), (3, 10)]
clusters, centers = cop_kmeans(dataset=input_matrix, k=4, ml=must_link,cl=cannot_link) 

In [482]:
import pandas as pd
df = pd.read_excel("/Users/berkbatuhangurhan/Desktop/Tesi/copsinecosine.xlsx", sheet_name=0)

In [483]:
person=df['Person']
person

0      P1
1      p1
2      p1
3      P1
4      P1
       ..
206    P2
207    P2
208    P2
209    P2
210    P2
Name: Person, Length: 211, dtype: object

In [484]:
hour = df['Hour'] + df['Minute']/60
df['sin'] = num.sin(hour*(2.*num.pi/24))
df['cos'] = num.cos(hour*(2.*num.pi/24))

In [485]:
df = df.drop(['Minute','Hour','Person'], 1)

a = df.to_numpy()

In [486]:
df

,WeekDay,Sensor,sin,cos
0,2,38,0.803857,-0.594823
1,2,37,0.790690,-0.612217
2,2,29,0.790690,-0.612217
3,2,43,0.790690,-0.612217
4,2,44,0.790690,-0.612217
...,...,...,...,...
206,6,38,0.737277,-0.675590
207,6,37,0.734323,-0.678801
208,6,29,0.728371,-0.685183
209,6,38,0.725374,-0.688355


In [487]:
a

array([[ 2.00000000e+00,  3.80000000e+01,  8.03856861e-01,
        -5.94822787e-01],
       [ 2.00000000e+00,  3.70000000e+01,  7.90689574e-01,
        -6.12217280e-01],
       [ 2.00000000e+00,  2.90000000e+01,  7.90689574e-01,
        -6.12217280e-01],
       [ 2.00000000e+00,  4.30000000e+01,  7.90689574e-01,
        -6.12217280e-01],
       [ 2.00000000e+00,  4.40000000e+01,  7.90689574e-01,
        -6.12217280e-01],
       [ 2.00000000e+00,  4.50000000e+01,  7.88010754e-01,
        -6.15661475e-01],
       [ 2.00000000e+00,  5.00000000e+01,  7.88010754e-01,
        -6.15661475e-01],
       [ 2.00000000e+00,  4.90000000e+01,  7.85316931e-01,
        -6.19093949e-01],
       [ 2.00000000e+00,  5.00000000e+01,  7.85316931e-01,
        -6.19093949e-01],
       [ 2.00000000e+00,  4.40000000e+01,  7.85316931e-01,
        -6.19093949e-01],
       [ 2.00000000e+00,  4.40000000e+01,  7.82608157e-01,
        -6.22514637e-01],
       [ 2.00000000e+00,  5.00000000e+01,  7.82608157e-01,
      

In [488]:
#cannot_link = [(6, 26), (8, 27), (9, 28), (10, 29), (11, 30), (10, 58), (11, 58), (5, 59), (6, 59), (7, 59), (8, 59), (9,59), (10, 59),(11, 59)]

#cannot_link=[(0,18),(5,19),(7,20),(12,24),(16,25),]

must_link=[(2,3),(2,5),(8,10),(14,18),(23,24),(21,22),(28,29),(36,37)
          ,(39,49),(43,44),(45,47),(50,54),(50,53),(55,59),(62,64),(65,66),(70,76),(79,80),(81,83),(84,86),
           (89,92),(99,101),(104,105),(109,112),(114,115),(121,122),(132,133),
          (136,139),(147,148),(150,151),(153,154),(158,159),(162,163),(163,164),(165,168),(171,172)
          ,(173,174),(175,176),(177,178),(181,182),(184,185),(191,193),(196,197),(200,202),(206,207)
          ]
cannot_link=[(6, 26), (8, 27), (9, 28), (10, 29), (11, 30),(16,33),(18,33),(20,34),(22,37)
            ,(45,55),(47,59)
            ,(60,81)
            ,(84,100),(88,103),(96,117)
            ,(123,141),(124,141),(121,136)
            ,(155,160),(155,161),(156,185)
            ,(186,191),(186,193),(189,196),(189,197)
            ,(200,209)]



In [489]:
cannot_link

[(6, 26),
 (8, 27),
 (9, 28),
 (10, 29),
 (11, 30),
 (16, 33),
 (18, 33),
 (20, 34),
 (22, 37),
 (45, 55),
 (47, 59),
 (60, 81),
 (84, 100),
 (88, 103),
 (96, 117),
 (123, 141),
 (124, 141),
 (121, 136),
 (155, 160),
 (155, 161),
 (156, 185),
 (186, 191),
 (186, 193),
 (189, 196),
 (189, 197),
 (200, 209)]

In [490]:
import numpy
#must_link = [(0,1),(0,2),(0,3),(4,5),(15,24)]
#cannot_link = [(5, 58), (6, 58), (7, 58), (8, 58), (9, 58), (10, 58), (11, 58), (5, 59), (6, 59), (7, 59), (8, 59), (9,59), (10, 59), (11, 59)]
clusters1, centers1 = cop_kmeans(dataset=a, k=2, ml=must_link,cl=cannot_link)

In [491]:
count_arr = num.bincount(clusters1)
print(count_arr[1])

100


In [497]:
countp1=0
countp1not=0
countp2=0
countp2not=0
for x in range(len(clusters1)):
    if person[x]=='P1' or person[x]=='p1':
        if clusters1[x]==1:
            countp1=countp1+1
        else:
            countp1not=countp1not+1
    if person[x]=='P2' or person[x]=='p2':
        if clusters1[x]==0:
            countp2=countp2+1
        else:
            countp2not=countp2not+1

In [498]:
print("P1 True Positive: ",countp1,", P1 False Positive",countp1not)

P1 True Positive:  87 , P1 False Positive 28


In [499]:
print("P2 True Positive: ",countp2,", P2 False Positive",countp2not)

P2 True Positive:  83 , P2 False Positive 13


In [500]:
clusters1


[1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1]

In [501]:
prec1= countp1 / (countp1+countp2not)
print("Precision of P1: ",prec1)

Precision of P1:  0.87


In [502]:
prec2= countp2 / (countp2+countp1not)
print("Precision of P2: ",prec2)

Precision of P2:  0.7477477477477478
